<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ejercicio de clasificación con redes neuronales (antes KNN)

Ejemplo de clasificación utilizando redes neuronales para la clasificación de drogadas que debería tomar un pasiente según su historial clínico<br>

v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import keras
from keras.models import Sequential
#from keras.utils import to_categorical  
from keras.utils.np_utils import to_categorical # Si esto no funciona, probar con el import anterior

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
if os.access('Telecust1.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/Telecust1.csv > Telecust1.csv
    else:
        !wget Telecust1.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/Telecust1.csv

### `Telecust1.csv.csv`:
El dataset **`Telecust1.csv.csv`** contiene diferentes tipos de clientes que consumen un servicio de telecomunicación, los cuales deseamos clasificar en 4 categorias.<br> [Dataset source](https://www.kaggle.com/prathamtripathi/customersegmentation)

- **region** --> region, ejemplo 2
- **tenure** --> grado permanencia, ejemplo 40
- **age** --> edad, ejemplo 52
- **income** --> ingresos o sueldo, ejemplo 50 (un número que no representa una moneda real
- **marital** --> si está casado o no
- **address** --> dirección
- **employ** --> empleo
- **retire** --> si está retirado o no
- **genero** --> 0 o 1 (no se sabe cual es cual)


# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
df = pd.read_csv("Telecust1.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
df.head()

In [ ]:
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
df.describe()

In [ ]:
# Exploramos que tan balanceado está el dataset,
# como está repartida las categorias entre los clientes actuales
df['custcat'].value_counts()

In [ ]:
ax = sns.countplot(data=df, x="custcat")

Se puede ver que el dataset está bastante balanceado, no habrá una tendencia marcada

In [ ]:
# Nos quedamos con aquellas columnas que podemos entender su relacion con el objetivo
df_clean = df[['tenure', 'age', 'income', 'marital', 'retire', 'gender', 'custcat']]
df_clean

#### Normalización de los datos

Analizar cual es la distribución de los datos numéricos
- tenure
- age
- income

In [ ]:
sns.displot(data=df_clean, x='age')

In [ ]:
sns.displot(data=df_clean, x='tenure')

In [ ]:
sns.displot(data=df_clean, x='income')

El "ingreso" sigue una distribución normal pero con muchos outliers, es por eso que no utilizaremos el MinMaxScaler sino que se utilizará el StandardScaler a pesar de que "tenure" no siga una distribución normal

In [ ]:
from sklearn.preprocessing import StandardScaler
df_norm = df_clean.copy()
age_scaler = StandardScaler()
tenure_scaler = StandardScaler()
income_scaler = StandardScaler()
df_norm.loc[:, 'age'] = age_scaler.fit_transform(df[['age']])
df_norm.loc[:, 'tenure'] = tenure_scaler.fit_transform(df[['tenure']])
df_norm.loc[:, 'income'] = income_scaler.fit_transform(df[['income']])
df_norm.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_enc = df_norm.copy()
# Reemplazar la columna de salida texto a números
df_enc['custcat'] = le.fit_transform(df_enc['custcat'])
# Clases identificados
le.classes_

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
X = df_enc.drop('custcat', axis=1).values
y = to_categorical(df_enc['custcat'])

In [ ]:
in_shape = X.shape[1]
in_shape

In [ ]:
out_shape = y.shape[1]
out_shape

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 80%20%

In [ ]:
from sklearn.model_selection import train_test_split
# Fijamos un "random_state" constante para que siempre el dataset se parta de la misma forma
# para poder repetir los ensayos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras.layers import Dense

def create_model(in_shape, hidden_neurons, out_shape):
    # Crear un modelo secuencial
    model = Sequential()

    # Crear la capa de entrada y la capa oculta (hidden) de la red, que tendrá:
    # --> tantas entradas (in_shape) como columnas
    # --> tantas neuronas como deseemos
    # --> utilizamos "sigmoid" como capa de activación
    model.add(Dense(units=hidden_neurons, activation='sigmoid', input_shape=(in_shape,)))

    # Crear la capa de salida, que tendrá tantas neuronas como salidas posibles
    model.add(Dense(units=out_shape, activation='softmax'))

    return model

In [ ]:
model = create_model(in_shape, 16, out_shape)

model.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_split=0.2 , epochs=50, batch_size=32)

In [ ]:
epoch_count = range(1, len(history.history['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=history.history['accuracy'], label='train')
sns.lineplot(x=epoch_count,  y=history.history['val_accuracy'], label='valid')
plt.show()

In [ ]:
y_hat_prob = model.predict(X_test)
y_hat = np.argmax(y_hat_prob,axis=1)

In [ ]:
model.summary()

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy)
#scores = model.evaluate(X_test, y_test, verbose=0)
scores = model.evaluate(X_test, y_test)
scores[1]

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test.argmax(axis=1), y_hat, normalize=True)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test.argmax(axis=1), y_hat)
cmd = ConfusionMatrixDisplay(cm, display_labels=le.classes_)
cmd.plot(cmap=plt.cm.Blues)
plt.show()

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
# Supongamos que deseamos ver a que categoría pertenecemos
# dado los siguientes datos
age = 25
tenure = 4
income = df['income'].mean() # ganamos el promedio
marital = 0 # no estamos casados
retire = 0 # no estamos retirados
gender = 1 # solo algun genero

In [ ]:
# El scaler espera como entrada una matriz (filas y columnas)
# Por eso el doble corchete
age_numpy = np.array([[age]])
# Utilizamos float para convertir la matriz que retorna el scaler
# a un número
age_norm = float(age_scaler.transform(age_numpy))
tenure_norm = float(tenure_scaler.transform(np.array([[tenure]])))
income_norm = float(income_scaler.transform(np.array([[income]])))
# El sistema espera como entrada "X" en este caso una sola fila pero varias
# columnas, por eso el reshape(1, -1) donde el "-1" significa "varias"
# (el sistema determina cuantas)
X_prueba = np.array([tenure_norm, age_norm, income_norm, marital, retire, gender]).reshape(1, -1)
print('Shape:', X_prueba.shape)
print('Valores:\n', X_prueba)

In [ ]:
mi_categoria_probabilidades = model.predict(X_prueba)
mi_categoria_probabilidades

In [ ]:
mi_categoria_dict = dict(zip(le.classes_, mi_categoria_probabilidades.flatten()))
mi_categoria_dict

In [ ]:
mi_categoria_ordenadas = sorted(mi_categoria_dict.items(), key=lambda x: x[1], reverse=True)
for categoria in mi_categoria_ordenadas:
    print(f'Grado de pertenencia a la categoria {categoria[0]}: {categoria[1]*100:.1f}%')

In [ ]:
category = le.inverse_transform((mi_categoria_probabilidades.argmax(),))
category

In [ ]:
category[0]

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

En este ejemplo se obtuvo muy poca performance, pero se obtuvo un mejor resultado que la vez que se aplicado el algoritmo de clasificación KNN en donde el accuracy obtenido fue del 34%